In [ ]:
""" 
Сравнение Clickhouse и Vertica

Используемый датасет: https://www.kaggle.com/datasets/souhagaa/nasa-access-log-dataset-1995?select=data.csv
"""


In [151]:
import os
import pandas as pd

data = pd.read_csv('data.csv', delimiter = ',', nrows=100000)
data.rename(columns={'Unnamed: 0':'id'}, inplace=True)
data = data.drop('bytes', axis=1)
data = data.drop('url', axis=1)

In [158]:
import vertica_python

connection_info = {
    'host': '127.0.0.1',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
}

def drop_table():
    with vertica_python.connect(**connection_info) as connection:
        cursor = connection.cursor()
        cursor.execute('DROP TABLE test_table')
    

def create_table():
    with vertica_python.connect(**connection_info) as connection:
        cursor = connection.cursor()
    
        cursor.execute("""
        CREATE TABLE test_table (
            id INTEGER,
            host VARCHAR(256) NOT NULL,
            time INTEGER NOT NULL,
            method VARCHAR(12) NOT NULL,
            response INTEGER NOT NULL
        );
        """)

insert_data = [tuple(i) for i in data.values.tolist()[:1000]]
def insert():
    with vertica_python.connect(**connection_info) as connection:
        cursor = connection.cursor()

        cursor.executemany("""
        INSERT INTO test_table (id, host, time, method, response) VALUES (%s, %s, %s, %s, %s);
        """, insert_data)

def select():
    with vertica_python.connect(**connection_info) as connection:
        cursor = connection.cursor()

        cursor.execute("""
            SELECT * FROM test_table;
        """)

drop_table()
create_table()

In [159]:
%timeit insert()

30.6 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [160]:
%timeit select()

14.4 ms ± 599 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [154]:
from clickhouse_driver import Client

client = Client(host='localhost')

In [163]:
client.execute('CREATE DATABASE IF NOT EXISTS example')
client.execute('DROP TABLE IF EXISTS example.test')
client.execute('''
        CREATE TABLE IF NOT EXISTS example.test (
        id Int64, host String, time Int64, method String, response Int32)
        Engine=MergeTree() ORDER BY id
''')

[]

In [164]:
insert_data = data.values.tolist()[0:1000]
%timeit client.execute('INSERT INTO example.test (id, host, time, method, response) VALUES', params=insert_data) 

14.5 ms ± 1.73 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [165]:
%timeit client.execute('SELECT * FROM example.test')

1.24 s ± 326 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
